# Exercise 3: Advanced Information Retrieval

## Question 1 - Latent Semantic Indexing


In this exercise, we will run latent semantic indexing on a term-document matrix using python numpy library.

Suppose we are given the following term-document matrix containing eleven terms and four documents $d_1$ , $d_2$ , $d_3$ and $d_4$:

$
M =
  \begin{bmatrix}
    d_1 & d_2 & d_3 & d_4 \\ 
	1 & 1 & 1 & 1  \\
	0 & 1 & 1 & 1 \\
	1 & 0 & 0 & 0 \\
	0 & 1 & 0 & 0 \\
    1 & 0 & 0 & 0 \\
    1 & 0 & 1 & 2 \\
    1 & 1 & 1 & 1 \\
    1 & 1 & 1 & 0 \\
    1 & 0 & 0 & 0 \\
    0 & 2 & 1 & 1 \\
    0 & 1 & 1 & 0 \\
  \end{bmatrix}
$


<br>

###  Question 1.a

Compute the singular value decomposition of the term-document matrix M. Print the values of the output matrices $K$, $S$ and $D^t$.


<b>Hint:</b> Use the function numpy.linalg.svd. More details of this function can be found here at this link:

https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.svd.html


Here's sample code:

In [2]:
# import Python matrix operations library
import numpy as np

#set M matrix using the given values.
M = [[1,1,1,1], 
     [0,1,1,1],
     [1,0,0,0],
     [0,1,0,0],
     [1,0,0,0],
     [1,0,1,2],
     [1,1,1,1],
     [1,1,1,0],
     [1,0,0,0],
     [0,2,1,1],
     [0,1,1,0]]


M = np.array(M)

# compute SVD
K,s,Dt = np.linalg.svd(M)


###  Question 1.b

Are the values of $S$ sorted? Perform latent semantic indexing by selecting the first two largest singular values of the matrix $S$.

<b>Hint:</b> See the lecture slides on latent semantic indexing for more details. A sub-matrix of a numpy matrix can be computed using indexing operations (see https://docs.scipy.org/doc/numpy-1.13.0/reference/arrays.indexing.html).


In [18]:
# watch out these dimensions are nxr and rxr and rxk
# hence watch out for d
K_sel = K[:,0:2]
S_sel = np.diag(s[0:2])
Dt_sel = Dt[0:2,:]

###  Question 1.c

Given the query $q$:

$
q =
  \begin{bmatrix}
	0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 1 \\ 0 \\ 0 \\ 0 \\ 1 \\ 1 \\
  \end{bmatrix}
$


Map query $q$ into the new document space $D$. The new query is referred to as $q^*$. 

<b>Hint:</b> Use the formulation for mapping queries provided in the lecture slides. You can also use np.linalg.inv function for computing the inverse of a matrix.

###  Question 1.d

Arrange the documents based on the cosine similarity measure between $q^*$ and the new documents in the space $D$.

<b>Hint:</b> Use the cosine similarity function from the previous exercise on vector space retrieval.

###  Question 1.e

Does the order of documents change if document $d_3$ is dropped? If yes, why? 
If no, how should $d_3$ be modified to change the document ordering?


### Question 1.f [Optional]

Run latent semantic indexing for the document collection presented in the previous exercise (presented here as well):

  DocID | Document Text
  ------|------------------
  1     | How to Bake Breads Without Baking Recipes
  2     | Smith Pies: Best Pies in London
  3     | Numerical Recipes: The Art of Scientific Computing
  4     | Breads, Pastries, Pies, and Cakes: Quantity Baking Recipes
  5     | Pastry: A Book of Best French Pastry Recipes

Now, for the query $Q=$''<i>baking</i>'', find the top ranked documents according to LSI (use three singular values). 

<b>Hint:</b> Use the code for computing document_vectors from the last exercise. However note that document_vectors represent document-term matrix whereas LSI uses term-document matrix.

In [19]:
# 1c
def create_query(q,S_sel,K_sel):
    q = np.array(q)
    inverse = np.linalg.inv(S_sel)
    q_lowDim = np.dot(np.dot(q.T,K_sel),inverse)
    return q_lowDim

In [35]:
# 1d
def cosine_similarity(vecA,vecB):
    prod = np.dot(vecA,vecB)
    sizeA = np.linalg.norm(vecA,2)
    sizeB = np.linalg.norm(vecB,2)
    res = (prod/(sizeA * sizeB))
    return res[0][0]


def query_system(Dt_sel,q_lowDim):
    D_temp = Dt_sel
    similarities = []
    for col in range(0,D_temp.shape[1]):
    
       
        sim = cosine_similarity(q_lowDim,D_temp[:,col].reshape(-1,1))
        similarities.append((col,sim))
    
    return sorted(similarities,key = lambda q : q[1])

In [36]:
# original
q = [[0], 
     [0],
     [0],
     [0],
     [0],
     [1],
     [0],
     [0],
     [0],
     [1],
     [1]]
q_lowDim = create_query(q,S_sel,K_sel)
print(query_system(Dt_sel,q_lowDim))

[(0, -0.012057913278690479), (3, 0.5931086268074781), (1, 0.9388827727147446), (2, 0.9524776244205609)]


In [38]:
#1d withot doc 3
drop_doc = np.delete(M,2,0)
K,s,Dt = np.linalg.svd(drop_doc)
K_sel = K[:,0:2]
S_sel = np.diag(s[0:2])
Dt_sel = Dt[0:2,:]
q = [[0], 
     [0],
     [0],
     [0],
     [1],
     [0],
     [0],
     [0],
     [1],
     [1]]
q_lowDim = create_query(q,S_sel,K_sel)
print(query_system(Dt_sel,q_lowDim))

[(0, 0.17395298953871513), (3, 0.5454101580670725), (1, 0.8644418052549182), (2, 0.9723182007460264)]
